In [21]:
!ls /teamspace/studios/this_studio

DataPreprocessing.ipynb  SepsisData.csv  Untitled1.ipynb  getting_started.ipynb
FT_Sepsis.ipynb		 Untitled.ipynb  examples


In [22]:
# # Install nbconvert if not already installed
!pip install nbconvert

# # Convert the current notebook to HTML
!jupyter nbconvert --to html "/teamspace/studios/this_studio/FT_Sepsis.ipynb"

[NbConvertApp] Converting notebook /teamspace/studios/this_studio/FT_Sepsis.ipynb to html
[NbConvertApp] Writing 393774 bytes to /teamspace/studios/this_studio/FT_Sepsis.html


In [6]:
!pip install imblearn
!pip install lib
!pip install torchsummary

In [1]:
# Import libraries
import math
import warnings
from typing import Dict, Literal
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.stats import zscore
from imblearn.over_sampling import SMOTE
import torch
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.std import tqdm
warnings.resetwarnings()
import lib
import torchsummary
!pip install torchinfo
from torchinfo import summary
!pip install pytorch-ignite
from ignite.handlers import EarlyStopping
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [2]:
# Load dataset
df = pd.read_csv('./SepsisData.csv', header=None, low_memory=False)

# Step 1: Drop the first row (original header) and set a new header
df = df.drop(index=0)
df.columns = df.iloc[0]
df = df.drop(index=1)

# Define the columns based on the categories
numeric_features = [
    'age', 'BMI', 'gcs', 'sirs', 'apsiii', 'lods', 'oasis', 'sapsii', 'sofa_total',
    'sofa_respiration', 'sofa_coagulation', 'sofa_liver', 'sofa_cardiovascular',
    'sofa_cns', 'sofa_renal', 'urineoutput_1stday', 'hematocrit_min', 'hematocrit_max',
    'hemoglobin_min', 'hemoglobin_max', 'platelets_min', 'platelets_max', 'wbc_min',
    'wbc_max', 'albumin_min', 'albumin_max', 'aniongap_min', 'aniongap_max', 'bicarbonate_min',
    'bicarbonate_max', 'calcium_min', 'calcium_max', 'chloride_min', 'chloride_max',
    'glucose_mean', 'sodium_min', 'sodium_max', 'potassium_min', 'potassium_max', 'bun_max',
    'creatinine_max', 'INR_min', 'INR_max', 'PT_min', 'PT_max', 'ptt_min', 'ptt_max',
    'ALT_max', 'ALP_max', 'AST_max', 'bilirubin_total_max', 'ld_ldh_max', 'heart_rate_max',
    'SBP_mean', 'DBP_mean', 'mbp_mean', 'resp_rate_min', 'resp_rate_max', 'temperature_min',
    'temperature_max', 'SpO2_min', 'lactate_max_bg', 'pCO2_min_bg', 'pCO2_max_bg',
    'baseexcess_min_bg', 'baseexcess_max_bg'
]

categorical_features = [
    'gender_M1F0', 'Myocardial_infarction', 'Congestive_heart_failure', 'Peripheral_vascular_disease',
    'Cerebrovascular_disease', 'Dementia', 'Chronic_pulmonary_disease', 'Rheumatic_disease',
    'peptic_ulcer_disease', 'mild_liver_disease', 'Diabetes', 'Hemiplegia_paraplegia',
    'renal_disease', 'malignancy', 'Moderate_or_severe_liver_disease', 'Metastatic_solid_tumor',
    'AIDS', 'vasoactive drug ', 'dobutamine', 'vasopressin', 'phenylephrine', 'norepinephrine',
    'dopamine', 'milrinone', 'epinephrine', 'MV'
]

label_encoding_feature = 'race'
output_features = ['death_28day', 'death_90day', 'death_1year']

# Step 2: Handle missing values
# Fill numeric columns with mean
for col in numeric_features:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Ensure numeric dtype
    df[col] = df[col].fillna(df[col].mean())
    df[col] = df[col].astype('int64')

# Fill categorical columns with mode
for col in categorical_features:
    df[col] = df[col].fillna(df[col].mode()[0])
    df[col] = df[col].astype('int64')

# Step 3: Label encode the 'race' column
label_encoder = LabelEncoder()
df[label_encoding_feature] = label_encoder.fit_transform(df[label_encoding_feature].fillna(df[label_encoding_feature].mode()[0]))

for col in output_features:
    df[col] = df[col].astype('int64')

# Step 4: Normalize all numeric columns using z-score
for col in numeric_features:
    df[col] = zscore(df[col])

# Step 5: Extract X (features) and Y (target)
Y = df['death_1year']  # Target column
X = df.drop(columns=['death_1year', 'death_28day', 'death_90day','MV'])  # Features

#Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Use RandomUnderSampler for imbalanced dataset
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_train_resampled, Y_train_resampled = rus.fit_resample(X_train, Y_train)
categorical_features.pop(25)
# print("X_train_resampled shape:", X_train_resampled.shape)
# print("Y_train_resampled shape:", Y_train_resampled.shape)
# Split training data into training and validation sets (e.g., 80% train, 20% val)
X_train_resampled, X_val, Y_train_resampled, Y_val = train_test_split(
    X_train_resampled, Y_train_resampled, test_size=0.2, random_state=42
)

# Convert the DataFrames to NumPy arrays before converting them to PyTorch tensors
X_train_tensor = torch.tensor(X_train_resampled.to_numpy(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train_resampled.to_numpy(), dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test.to_numpy(), dtype=torch.long)

X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
Y_val_tensor = torch.tensor(Y_val.to_numpy(), dtype=torch.long)

In [3]:
# Count the number of rows with label 0 and label 1 in the training set
label_counts = df['death_1year'].value_counts()
print("Label Counts:")
print(label_counts)

# print("X_train_resampled shape:", X_train_resampled.shape)
# print("Y_train_resampled shape:", Y_train_resampled.shape)

print("X_train_tensor shape:", X_train_tensor.shape)
print("Y_train_tensor shape:", Y_train_tensor.shape)
print("X_val_tensor shape:", X_val_tensor.shape)
print("Y_val_tensor shape:", Y_val_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)
print("Y_test_tensor shape:", Y_test_tensor.shape)


# Check the count of each label after undersampling
label_counts = pd.Series(Y_train_resampled).value_counts()
print("Count of each label after undersampling:")
print(label_counts)

label_counts_testset = pd.Series(Y_test_tensor).value_counts()
print("Count of each label in testset:")
print(label_counts_testset)

Label Counts:
death_1year
0    14570
1     4230
Name: count, dtype: int64
X_train_tensor shape: torch.Size([5414, 92])
Y_train_tensor shape: torch.Size([5414])
X_val_tensor shape: torch.Size([1354, 92])
Y_val_tensor shape: torch.Size([1354])
X_test_tensor shape: torch.Size([3760, 92])
Y_test_tensor shape: torch.Size([3760])
Count of each label after undersampling:
death_1year
1    2719
0    2695
Name: count, dtype: int64
Count of each label in testset:
0    2914
1     846
Name: count, dtype: int64


In [4]:
# Ensure that all columns are encoded as needed and converted to PyTorch tensors

# Separate numerical and categorical features
X_train_num = torch.tensor(X_train_resampled[numeric_features].to_numpy(), dtype=torch.float32)
X_train_cat = torch.tensor(X_train_resampled[categorical_features].to_numpy(), dtype=torch.long)

X_test_num = torch.tensor(X_test[numeric_features].to_numpy(), dtype=torch.float32)
X_test_cat = torch.tensor(X_test[categorical_features].to_numpy(), dtype=torch.long)

# Target labels
Y_train_tensor = torch.tensor(Y_train_resampled.to_numpy(), dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test.to_numpy(), dtype=torch.long)


In [5]:
print(X_train_num)
print(X_train_num.shape)

print(X_train_cat)
print(X_train_cat.shape)

tensor([[ 0.5826, -1.0166,  0.5710,  ..., -0.0359,  0.1061, -0.0194],
        [ 0.2142,  0.0081,  0.2421,  ...,  1.5111,  0.5907,  2.0411],
        [ 1.5650,  0.0081, -2.0604,  ..., -0.0359,  0.1061, -0.0194],
        ...,
        [ 0.5826,  0.6912,  0.2421,  ..., -0.0359,  0.1061, -0.0194],
        [ 1.5650,  0.0081, -0.0869,  ..., -1.0369,  1.0753,  0.2750],
        [-0.2770,  0.1789, -1.0736,  ...,  4.0592,  3.7408,  4.1016]])
torch.Size([5414, 66])
tensor([[1, 1, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
torch.Size([5414, 25])


In [6]:
import torch.nn.functional as F
# %%
import math
import typing as ty
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as nn_init
# import zero
from torch import Tensor

import lib


# %%
class Tokenizer(nn.Module):
    category_offsets: ty.Optional[Tensor]

    def __init__(
        self,
        d_numerical: int,       # Dimensionality of numerical features.
        categories: ty.Optional[ty.List[int]],  # Cardinalities of categorical features.
        d_token: int, # Dimensionality of token embeddings.
        bias: bool,  # Whether to include a bias term.
    ) -> None:
        super().__init__()
        if categories is None:
            d_bias = d_numerical
            self.category_offsets = None
            self.category_embeddings = None
        else:
            d_bias = d_numerical + len(categories)
            category_offsets = torch.tensor([0] + categories[:-1]).cumsum(0)
            self.register_buffer('category_offsets', category_offsets)
            self.category_embeddings = nn.Embedding(sum(categories), d_token)
            nn_init.kaiming_uniform_(self.category_embeddings.weight, a=math.sqrt(5))
            print(f'{self.category_embeddings.weight.shape=}')

        # take [CLS] token into account
        self.weight = nn.Parameter(Tensor(d_numerical + 1, d_token))  #Parameter matrix for embedding numerical features plus a [CLS] token
        self.bias = nn.Parameter(Tensor(d_bias, d_token)) if bias else None #Optional bias matrix for adjusting the final embeddings
        # The initialization is inspired by nn.Linear
        nn_init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            nn_init.kaiming_uniform_(self.bias, a=math.sqrt(5))

    @property
    def n_tokens(self) -> int:
        return len(self.weight) + (
            0 if self.category_offsets is None else len(self.category_offsets)
        )

    def forward(self, x_num: Tensor, x_cat: ty.Optional[Tensor]) -> Tensor:
        x_some = x_num if x_cat is None else x_cat
        assert x_some is not None
        x_num = torch.cat(
            [torch.ones(len(x_some), 1, device=x_some.device)]  # [CLS] Adds a [CLS] token (a column of ones) to x_num
            + ([] if x_num is None else [x_num]),
            dim=1,
        )
        x = self.weight[None] * x_num[:, :, None] #Computes embeddings for numerical features using self.weight
        if x_cat is not None:
            x = torch.cat(
                [x, self.category_embeddings(x_cat + self.category_offsets[None])],
                dim=1,
            ) #Concatenates categorical and numerical embeddings along the feature dimension
        if self.bias is not None:
            bias = torch.cat(
                [
                    torch.zeros(1, self.bias.shape[1], device=x.device),
                    self.bias,
                ]
            )
            x = x + bias[None]
        return x  #Returns the combined numerical and categorical token embeddings for the batch


class MultiheadAttention(nn.Module):
    def __init__(
        self, d: int, n_heads: int, dropout: float, initialization: str
    ) -> None:
        if n_heads > 1:
            assert d % n_heads == 0
        assert initialization in ['xavier', 'kaiming']

        super().__init__()
        self.W_q = nn.Linear(d, d)
        self.W_k = nn.Linear(d, d)
        self.W_v = nn.Linear(d, d)
        self.W_out = nn.Linear(d, d) if n_heads > 1 else None
        self.n_heads = n_heads
        self.dropout = nn.Dropout(dropout) if dropout else None

        for m in [self.W_q, self.W_k, self.W_v]:
            if initialization == 'xavier' and (n_heads > 1 or m is not self.W_v):
                # gain is needed since W_qkv is represented with 3 separate layers
                nn_init.xavier_uniform_(m.weight, gain=1 / math.sqrt(2))
            nn_init.zeros_(m.bias)
        if self.W_out is not None:
            nn_init.zeros_(self.W_out.bias)

    def _reshape(self, x: Tensor) -> Tensor:
        batch_size, n_tokens, d = x.shape
        d_head = d // self.n_heads
        return (
            x.reshape(batch_size, n_tokens, self.n_heads, d_head)
            .transpose(1, 2)
            .reshape(batch_size * self.n_heads, n_tokens, d_head)
        )

    def forward(
        self,
        x_q: Tensor,
        x_kv: Tensor,
        key_compression: ty.Optional[nn.Linear],
        value_compression: ty.Optional[nn.Linear],
    ) -> Tensor:
        q, k, v = self.W_q(x_q), self.W_k(x_kv), self.W_v(x_kv)
        for tensor in [q, k, v]:
            assert tensor.shape[-1] % self.n_heads == 0
        if key_compression is not None:
            assert value_compression is not None
            k = key_compression(k.transpose(1, 2)).transpose(1, 2)
            v = value_compression(v.transpose(1, 2)).transpose(1, 2)
        else:
            assert value_compression is None

        batch_size = len(q)
        d_head_key = k.shape[-1] // self.n_heads
        d_head_value = v.shape[-1] // self.n_heads
        n_q_tokens = q.shape[1]

        q = self._reshape(q)
        k = self._reshape(k)
        attention = F.softmax(q @ k.transpose(1, 2) / math.sqrt(d_head_key), dim=-1)
        if self.dropout is not None:
            attention = self.dropout(attention)
        x = attention @ self._reshape(v)
        x = (
            x.reshape(batch_size, self.n_heads, n_q_tokens, d_head_value)
            .transpose(1, 2)
            .reshape(batch_size, n_q_tokens, self.n_heads * d_head_value)
        )
        if self.W_out is not None:
            x = self.W_out(x)
        return x


class Transformer(nn.Module):
    """Transformer.

    References:
    - https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html
    - https://github.com/facebookresearch/pytext/tree/master/pytext/models/representations/transformer
    - https://github.com/pytorch/fairseq/blob/1bba712622b8ae4efb3eb793a8a40da386fe11d0/examples/linformer/linformer_src/modules/multihead_linear_attention.py#L19
    """

    def __init__(
        self,
        *,
        # tokenizer
        d_numerical: int,  # Number of numerical features.
        categories: ty.Optional[ty.List[int]],  # Cardinalities of categorical features.
        token_bias: bool,          # Whether to include bias in tokenizer embeddings.
        # transformer
        n_layers: int,             # Number of Transformer layers.
        d_token: int,              # Dimensionality of token embeddings.
        n_heads: int,              # Number of attention heads.
        d_ffn_factor: float,       # Feedforward layer dimensionality as a multiplier of `d_token`.
        attention_dropout: float,  # Dropout rate for attention.
        ffn_dropout: float,        # Dropout rate for feedforward layers.
        residual_dropout: float,   # Dropout rate for residual connections.
        activation: str,           # Activation function name (e.g., "relu").
        prenormalization: bool,    # Whether to apply layer normalization before operations.
        initialization: str,       # Initialization method ("xavier" or others).
        # linformer
        kv_compression: ty.Optional[float],  # Compression ratio for attention keys/values.
        kv_compression_sharing: ty.Optional[str],  # Sharing strategy for key/value compression.
        d_out: int,                # Output dimensionality.
    ) -> None:
        assert (kv_compression is None) ^ (kv_compression_sharing is not None)

        super().__init__()
        self.tokenizer = Tokenizer(d_numerical, categories, d_token, token_bias)
        n_tokens = self.tokenizer.n_tokens

        def make_kv_compression():
            assert kv_compression
            compression = nn.Linear(
                n_tokens, int(n_tokens * kv_compression), bias=False
            )
            if initialization == 'xavier':
                nn_init.xavier_uniform_(compression.weight)
            return compression

        self.shared_kv_compression = (
            make_kv_compression()
            if kv_compression and kv_compression_sharing == 'layerwise'
            else None
        )

        def make_normalization():
            return nn.LayerNorm(d_token)

        d_hidden = int(d_token * d_ffn_factor)
        self.layers = nn.ModuleList([])
        for layer_idx in range(n_layers):
            layer = nn.ModuleDict(
                {
                    'attention': MultiheadAttention(
                        d_token, n_heads, attention_dropout, initialization
                    ),
                    'linear0': nn.Linear(
                        d_token, d_hidden * (2 if activation.endswith('glu') else 1)
                    ),
                    'linear1': nn.Linear(d_hidden, d_token),
                    'norm1': make_normalization(),
                }
            )
            if not prenormalization or layer_idx:
                layer['norm0'] = make_normalization()
            if kv_compression and self.shared_kv_compression is None:
                layer['key_compression'] = make_kv_compression()
                if kv_compression_sharing == 'headwise':
                    layer['value_compression'] = make_kv_compression()
                else:
                    assert kv_compression_sharing == 'key-value'
            self.layers.append(layer)

        # Set activation functions directly in the Transformer class
        self.activation = F.relu  # or any other activation function, like torch.nn.ReLU()
        self.last_activation = F.relu  # or any other non-GLU activation
        # self.activation = lib.get_activation_fn(activation)
        # self.last_activation = lib.get_nonglu_activation_fn(activation)
        self.prenormalization = prenormalization
        self.last_normalization = make_normalization() if prenormalization else None
        self.ffn_dropout = ffn_dropout
        self.residual_dropout = residual_dropout
        self.head = nn.Linear(d_token, d_out)

    def _get_kv_compressions(self, layer):
        return (
            (self.shared_kv_compression, self.shared_kv_compression)
            if self.shared_kv_compression is not None
            else (layer['key_compression'], layer['value_compression'])
            if 'key_compression' in layer and 'value_compression' in layer
            else (layer['key_compression'], layer['key_compression'])
            if 'key_compression' in layer
            else (None, None)
        )

    def _start_residual(self, x, layer, norm_idx):
        x_residual = x
        if self.prenormalization:
            norm_key = f'norm{norm_idx}'
            if norm_key in layer:
                x_residual = layer[norm_key](x_residual)
        return x_residual

    def _end_residual(self, x, x_residual, layer, norm_idx):
        if self.residual_dropout:
            x_residual = F.dropout(x_residual, self.residual_dropout, self.training)
        x = x + x_residual
        if not self.prenormalization:
            x = layer[f'norm{norm_idx}'](x)
        return x

    def forward(self, x_num: Tensor, x_cat: ty.Optional[Tensor]) -> Tensor:
        x = self.tokenizer(x_num, x_cat)

        for layer_idx, layer in enumerate(self.layers):
            is_last_layer = layer_idx + 1 == len(self.layers)
            layer = ty.cast(ty.Dict[str, nn.Module], layer)

            x_residual = self._start_residual(x, layer, 0)
            x_residual = layer['attention'](
                # for the last attention, it is enough to process only [CLS]
                (x_residual[:, :1] if is_last_layer else x_residual),
                x_residual,
                *self._get_kv_compressions(layer),
            )
            if is_last_layer:
                x = x[:, : x_residual.shape[1]]
            x = self._end_residual(x, x_residual, layer, 0)

            x_residual = self._start_residual(x, layer, 1)
            x_residual = layer['linear0'](x_residual)
            x_residual = self.activation(x_residual)
            if self.ffn_dropout:
                x_residual = F.dropout(x_residual, self.ffn_dropout, self.training)
            x_residual = layer['linear1'](x_residual)
            x = self._end_residual(x, x_residual, layer, 1)

        assert x.shape[1] == 1  #x is a 3D tensor with shape [batch_size, seq_len, d_token]
        x = x[:, 0] #The [CLS] token is always the first token in the sequence,extracts the embedding of the [CLS] token for each sample in the batch
        if self.last_normalization is not None:
            x = self.last_normalization(x)
        x = self.last_activation(x)
        x = self.head(x)
        x = x.squeeze(-1)
        return x

In [7]:
import lib
# Define parameters for Model 1
d_numerical = len(numeric_features)  # Number of numerical features
categories = [X_train_resampled[cat].nunique() for cat in categorical_features]  # Number of unique values per categorical feature
d_token = 64  # Embedding size for each token; adjust as needed
n_heads = 4  # Number of attention heads
n_layers = 3  # Number of transformer layers
d_ffn_factor = 4.0  # Factor for feedforward network dimension
attention_dropout = 0.1
ffn_dropout = 0.1
residual_dropout = 0.1
activation = "relu"
prenormalization = True
initialization = "xavier"
token_bias = True
kv_compression = None  # or a float value for compression ratio
kv_compression_sharing = None  # adjust based on kv_compression
d_out = 1  # Number of output classes (e.g., for a classification problem)

# Initialize model
model = Transformer(
    d_numerical=d_numerical,
    categories=categories,
    token_bias=token_bias,
    n_layers=n_layers,
    d_token=d_token,
    n_heads=n_heads,
    d_ffn_factor=d_ffn_factor,
    attention_dropout=attention_dropout,
    ffn_dropout=ffn_dropout,
    residual_dropout=residual_dropout,
    activation=activation,
    prenormalization=prenormalization,
    initialization=initialization,
    kv_compression=kv_compression,
    kv_compression_sharing=kv_compression_sharing,
    d_out=d_out
)


self.category_embeddings.weight.shape=torch.Size([50, 64])


In [8]:
# Define loss and optimizer
criterion = F.binary_cross_entropy_with_logits
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    predictions = model(X_train_num, X_train_cat)

    Y_train_tensor = Y_train_tensor.float()
    # Compute loss
    loss = criterion(predictions, Y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}")


Epoch [1/100], Loss: 0.7297
Epoch [2/100], Loss: 0.7551
Epoch [3/100], Loss: 0.7009
Epoch [4/100], Loss: 0.6971
Epoch [5/100], Loss: 0.7023
Epoch [6/100], Loss: 0.7001
Epoch [7/100], Loss: 0.6961
Epoch [8/100], Loss: 0.6931
Epoch [9/100], Loss: 0.6936
Epoch [10/100], Loss: 0.6947
Epoch [11/100], Loss: 0.6949
Epoch [12/100], Loss: 0.6958
Epoch [13/100], Loss: 0.6942
Epoch [14/100], Loss: 0.6941
Epoch [15/100], Loss: 0.6933
Epoch [16/100], Loss: 0.6923
Epoch [17/100], Loss: 0.6925
Epoch [18/100], Loss: 0.6928
Epoch [19/100], Loss: 0.6934
Epoch [20/100], Loss: 0.6929
Epoch [21/100], Loss: 0.6918
Epoch [22/100], Loss: 0.6923
Epoch [23/100], Loss: 0.6913
Epoch [24/100], Loss: 0.6907
Epoch [25/100], Loss: 0.6911
Epoch [26/100], Loss: 0.6906
Epoch [27/100], Loss: 0.6911
Epoch [28/100], Loss: 0.6903
Epoch [29/100], Loss: 0.6909
Epoch [30/100], Loss: 0.6900
Epoch [31/100], Loss: 0.6898
Epoch [32/100], Loss: 0.6886
Epoch [33/100], Loss: 0.6866
Epoch [34/100], Loss: 0.6860
Epoch [35/100], Loss: 0

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
import torchsummary

# Helper function to compute evaluation metrics
def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    return accuracy, precision, recall, f1

# Display model summary
def model_summary(model, input_size):
    torchsummary.summary(model, input_size=input_size)

# Plot training and validation loss
def plot_loss(train_losses, val_losses=None):
    plt.plot(train_losses, label="Train Loss")
    if val_losses:
        plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()

In [10]:
is_binary_classification = True  # Set to True if binary classification, False otherwise

with torch.no_grad():
    test_predictions = model(X_test_num, X_test_cat)

    if is_binary_classification:
        # Binary classification
        test_predictions = torch.sigmoid(test_predictions)
        y_pred = (test_predictions > 0.5).long()  # Convert to binary
    else:
        # Multiclass classification
        y_pred = torch.argmax(test_predictions, dim=1)  # Get predicted class indices

# Now you can proceed with converting predictions and labels to CPU for metric calculations
# Convert predictions and labels to CPU for metric calculations
y_true = Y_test_tensor.cpu().numpy()
y_pred = y_pred.cpu().numpy()

# Compute metrics
accuracy, precision, recall, f1 = compute_metrics(y_true, y_pred)

# Display classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

Accuracy: 0.7266
Precision: 0.8165
Recall: 0.7266
F1 Score: 0.7479

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.70      0.80      2914
           1       0.44      0.80      0.57       846

    accuracy                           0.73      3760
   macro avg       0.68      0.75      0.68      3760
weighted avg       0.82      0.73      0.75      3760

